Das Notebook ist dafür da die User Stories auszuführen. Bitte testet sie davor in eurem .main (und macht euer main in das gitignore). Hier also nur funktionierende User Stories!

Damit der import richtig funktioniert müsst ihr die init datei des jeweiligen Ordners (model/data_access/business_logic) pflegen.

In [1]:
import os
os.environ["DB_FILE"] = os.path.abspath("database/hotel_reservation_sample.db")

import model
import data_access
import business_logic

Hotel_manager = business_logic.HotelManager()

1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit 
ich dasjenige auswählen kann, welches meinen Wünschen 
entspricht. Wünsche sind:

1.1. Ich möchte alle Hotels in einer Stadt durchsuchen, 
damit ich das Hotel nach meinem bevorzugten Standort 
(Stadt) auswählen kann.

In [2]:
by_city = Hotel_manager.get_hotels_by_city("Zürich")
print(by_city)

[Hotel(
  ID: 1
  Name: Hotel Baur au Lac
  Stars: 5
  Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
  Rooms: 0
)]


1.2. Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

In [3]:
by_city_and_stars = Hotel_manager.get_hotels_by_city_and_stars("Bern", 5)
print(by_city_and_stars)

[Hotel(
  ID: 4
  Name: Bellevue Palace
  Stars: 5
  Address ID: Address(
  ID: 4
  Street: Marktgasse 59
  City: Bern
  Zipcode: 3011
)
  Rooms: 0
)]


1.3. Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung).

In [4]:
by_max_guests = Hotel_manager.get_hotels_by_city_and_max_guests("Bern", 1)
print(by_max_guests)

No hotels found in Bern with a maximum of 1 guests


1.4. Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.

1.5. Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne.

1.6. Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [ ]:
show_user_friendly = Hotel_manager.read_hotel_userfriendly()
for hotels in show_user_friendly:
    print(hotels)

2. Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.